## INDICE PISOS RM

### 1. ESTRUCTURA DE LAS CARPETAS Y ARCHIVOS 
    
    1.1 Bases de datos
    1.2 Carpetas:  Modelo_data_sucios y Modelo_sin_feature_inge
    1.3 Orden de revision de los Notebook principales
  
### 2. EXPLICACIÓN DEL CÓDIGO DE LOS NOTEBOOK PRINCIPALES
    2.1 pisos_rm_clean
    2.2 pisos_rm_analisis
    2.3 pisos_rm_mode_1

#  1. ESTRUCTURA DE LAS CARPETAS Y ARCHIVOS 

## 1.1 Bases de datos

Todas las bases de datos necesarias para trabajar en cada uno de los notebook los encontraras en la carpeta data. 

## 1.2 Carpetas:  Modelo_data_sucios y Modelo_sin_feature_inge

En estas carpetas encontraras los modelos iniciales, y como bien indican sus nombres hacen alusión al proceso de aprendizaje y de la evolucion de los resultados. 

Dicho esto, no estan los resultados finales.

# 1.3 Orden de revision de los Notebook principales

1º pisos_rm_clean: En este notebook se realiza la primera exploración de los datos, la identificacion e eliminacion de errores de escritura y outlier.



2º pisos_rm_analisis: En este notebook analizaremos la distribucion de los datos de nuestras variables y del target, la correlación existente y posteriormente la union con las variables anexadas.   



3º pisos_modelo1: Habiendo realizado la limpieza de datos y feature inge. encontraremos en este notebook cada uno de los modelos probados, sus resultados respectivos, posteriormente identificaremos los dos mejores modelos, a los cuales aplicaremos pipeline y Gridsearch.

___
___


# 2. EXPLICACIÓN DEL CÓDIGO DE LOS NOTEBOOK PRINCIPALES

##  2.1 pisos_rm_clean

Exportamos la librerias correpondientes.

1) Extraemos los datos:

    df= pd.read_excel("data/pisos_rm.xlsx


2) Identificamos NaN, naturaleza de los valores 0 y tipo de datos por feature

    status(df).sort_values(by="q_nan",ascending=False)

3) Identificamos los valores máximos y mínimos de nuestro target

    df.max()
    df.min()

4) Revisamos si existen datos duplicados 

     df.duplicated().sum()  

5) Convertimos los valores de nuestro target en INT

    df["UF"]=df["UF"].astype(int)

6) Visualizamos la correlación de nuestras variables 

    sns.heatmap(df.corr(), annot=True)

7) Visualizamos la distribucion de los datos  
    
    sns.pairplot(df)

8) Normalizamos los valores de "UF", creando una nueva columna

    df["Uf_log"]=np.log1p(df["UF"])#luego al precio en "UF_log"
    
    Poeteriormente habrá que aplicar la funcion matematica inversa, back = np.expm1()

9) Separamos la columna dirección, creando la columna comuna

    df["Direccion"].str.split(",",expand= True)
    direc=df["Direccion"].str.split(",",expand= True)

    direc.drop(direc.columns[2:12],axis=1, inplace= True)

    direc= direc.rename(columns={0:"Comuna", 1:"direcc"})

10) Las unimos a nuestro dataframe 

    df=pd.concat([df,direc],axis=1)

11) Eliminamos la columna Direccion 

    df.drop(["Direccion"],axis=1)

ELIMINAMOS LOS ERRORES DE ESCRITURA Y OUTLIER

12) Eliminamos la columna m2 construidos (porque tiene los mismo valores que m2 totales )

    df.drop(["m2_Construidos"],axis=1)

13) ELiminamos todos los pisos con m2 totales igual a 0 

    df.loc[df["m2_totales"]>0]
    df=df.loc[df["m2_totales"]>0]

14) Eliminamos todos los pisos que valen menos de 20 millones de pesos

    df=df.loc[df["Precio"]>20000000]

15) Eliminamos todos los pisos que valen mas 15 * 10^8   

    df=df.loc[df["Precio"]<1500000000]

16) Eliminamos todos los pisos que tengan menos de 20 m2 

    df=df.loc[df["m2_totales"]>19]

17) Eliminamos todos los pisos que tengan mas de 5 estacionamientos, 5 baños y 6 habitacines

    df=df.loc[df["Estacionamientos"]<=5]
    df=df.loc[df["Banos"]<6]
    df=df.loc[df["Habitaciones"]<7]

 18) Vemos la nueva Distribucion    
    
    sns.pairplot(df)

19) guardamos nuestra nueva base de datos con todos los        cambios realizados

    df.to_excel("data/clean_pisos.xlsx", index = False)

## 2.2 pisos_rm_analisis

Exportamos las librerias correspondientes

1) cargamos nuestra base de datos 

    piso=pd.read_excel("data/clean_pisos.xlsx")

2) le cambiamos el nombre a nuestro target 

    piso.rename(columns={"Uf_log":"UF_log"},inplace=True)

3) Realizamos un análisis exploratorio de los datos depurados

    status(piso).sort_values(by="q_nan", ascending=False)
    piso.describe().T.round(0)
    piso.max()
    pisos.min()

4) Observamos la distribución de los datos con Histogramas

    def plot_hist(variable):
    plt.figure(figsize = (9,3))
    plt.hist(piso[variable], bins = 50 , rwidth=.85)
    plt.xlabel(variable)
    plt.ylabel("Frequency")
    plt.title("{} Distribución de los Datos con Histograma".format(variable))
    plt.show()

    numero_variable =['Precio', 'UF','UF_log', 'Habitaciones', 'Banos',
       'Estacionamientos']

    for n in numero_variable:
        plot_hist(n)

    fig, ax = plt.subplots()
    grf=piso["Comuna"].value_counts().plot(ax=ax, kind='bar',figsize = (12,6))    

5)  Vemos la nueva correlación de los datos 

    sns.heatmap(piso.corr(), annot=True)

6) Cargamos y revisamos las nuevas variables

    variables=pd.read_excel("data/numero_pisos.xlsx") 
    status(variables).sort_values(by="q_nan",ascending=False)    

7) Cambiamos el orden de la columnas para favorecer el cruce de datos con el dataframe variables  

    piso = piso[['Comuna','m2_totales', 'Habitaciones', 'Banos',"Estacionamientos", 'Publi','Precio', 'UF',"UF_log"]] 

8) Pasamos la columna comuna a indice para facilitar la union con el dataframe principal pisos 


    variables.set_index("Comuna",inplace=True)

9) Unimos los dos datagframe 

    union= piso.join(variables, on=["Comuna"], how= "inner")

10) revisamos la nueva correlacion con el target 

    union.corr()
    correlacion= union.corr()
    correlacion["UF"].sort_values(ascending=False)

11) visualizamos la distribución de las nuevas variables y la correlación con el target por categorias

Otros 

cols_to_plot1= ['UF_log',
       'Poblacion total 2020', 'Superficie de Área Verde m²',
       'Superficie Plazas m²',' luminarias cada 50 metros lineales ',
       ' Consumo Per Capita residencial (Kwh/persona)',
       'Fallos suministro electrico',]

       sns.heatmap(union[cols_to_plot1].corr(), annot=True)

Educación:

        cols_to_plot2= ['UF_log',
      
       'Distancia a Educación Inicial (m)',
       'Razón entre disponibilidad efectiva de matrículas y demanda potencial por educación básica',
       'Número de establecimientos eduacion basica',
       'Número de establecimientos privados',
       'Distancia a Educación Basica (m)',
      ]

      sns.pairplot(union[cols_to_plot2])

Salud:
        cols_to_plot3=["UF_log",'Número de establecimientos atencion primaria',
       'Distancia a Centro de Salud (m)','% fonasa (Prevision Salud)',
       '% ff.aa. y del orden (Prevision Salud)', '% isapre (Prevision Salud)',
       ' % ninguno (Prevision Salud)', '% otro sistema (Prevision Salud)',]

       sns.heatmap(union[cols_to_plot3].corr(), annot=True)

Infraestructura y servicios:

       cols_to_plot4=["UF_log",'porcentaje manzanas con veredas con buena calidad de pavimento',
       'Total de residuos (t)',
       ' Porcentaje de residuos municipales valorizados',
       ' Porcentaje de viviendas en mal estado  y/o carente  servicios básicos',
       'Porcentaje de participación del FCM en el Ingreso Municipal Total',
       'Tasa de conexiones residenciales fijas de internet por cada 1.000 viviendas particulares',
       'Longitud ciclovías (km)', 'Tiempo de viaje en trans publico hr punta','% de Acceso deficitario a Servicios Basicos ',]


        sns.heatmap(union[cols_to_plot4].corr(), annot=True)

 Pobreza y vulnerabilidad Social:

        cols_to_plot5 = ["UF_log",'Víctimas Robo con violencia o Intimidación',
       'Víctimas Robo por Sorpresa',
       ' Número de denuncias por delito en el espacio público cada 100 habitantes',
       ' Porcentaje de unidades vecinales que tienen entre 20% y 60% de hogares vulnerables',
       ' Porcentaje de la población en situación de pobreza (pobreza multidimensional MDSF)',
       'Porcentaje de Hacinamiento',
       'Porcentaje de viviendas con situación de allegamiento externo',
       ' Requerimiento de viviendas nuevas urbanas','Ingreso promedio del hogar por Comuna',] 

       sns.heatmap(union[cols_to_plot5].corr(), annot=True)

12) Transformamos cada comuna en una variable y las unimos al  dataframe principal

        dummis_comunas= pd.get_dummies(union["Comuna"])
        dummis_comunas.head()
        pisos=pd.concat([union,dummis_comunas],axis=1)

13) cambiamos el uint8 a int de cada una de las nuevas columnas, que son las comunas  

        le=LabelEncoder()


    for i in pisos.select_dtypes("uint8").keys(): #
        pisos[i]=le.fit_transform(pisos[i])

14) Guardamos nuestra nueva base de datos 

    pisos.to_excel("data/pisos_analis_data.xlsx", index = False)

## 2.3 pisos_rm_mode_1

Exportamos las librerias correspondientes

1) Extraemos los datos:

    pisos=pd.read_excel("data/pisos_analis_data.xlsx")

2) definimos las variables 

    X= pisos.drop(["Comuna",'Publi','Precio', 'UF', "UF_log",'Porcentaje de viviendas con situación de allegamiento externo','Víctimas Robo por Sorpresa', 'Número de establecimientos educacion inicial','Distancia a Educación Basica (m)'], axis=1)
    y= pisos["UF_log"]

    seed=12

3) Dividimos los datos en test y train 

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = seed)

    print("Total features shape:", X.shape)
    print("Train features shape:", X_train.shape)
    print("Train target shape:", y_train.shape)
    print("Test features shape:", X_test.shape)
    print("Test target shape:", y_test.shape)

4) Aplicamos modelos 


5) Aplicamos  las métricas a los dos mejores modelos y visualizamos el feature importance  

2º mejor modelo: from xgboost import XGBRegressor

    xgb_reg = XGBRegressor(n_estimators=50, random_state=seed)
    xgb_reg.fit(X_train, y_train)

    print("Score train :", metrics.r2_score(y_train, xgb_reg.predict(X_train)))
    print("Score test:", metrics.r2_score(y_test, xgb_reg.predict(X_test)))
    acc5 = metrics.r2_score(y_test, xgb_reg.predict(X_test))

    predictions = xgb_reg.predict(X_test)

    print('MAE:', metrics.mean_absolute_error(np.expm1(y_test), np.expm1(predictions)))  
    print('MSE:', metrics.mean_squared_error(np.expm1(y_test), np.expm1(predictions)))
    print('RMSE:', np.sqrt(metrics.mean_squared_error(np.expm1(y_test), np.expm1(predictions))))
    print('MAPE:', metrics.mean_absolute_percentage_error(np.expm1(y_test), np.expm1(predictions)))

    xgb_reg.feature_importances_
    df_impor_xgb=pd.DataFrame({"col":X_train.columns, "coef_impor":xgb_reg.feature_importances_})
    df_impor_xgb = df_impor_xgb.round(decimals=2)
    df_impor_xgb.sort_values(by= "coef_impor", ascending = False).head(10)



Mejor modelo: from sklearn.ensemble import GradientBoostingRegressor

    gbrt = GradientBoostingRegressor(n_estimators=300, random_state=seed,max_depth=3)
    gbrt.fit(X_train, y_train)

    print("Score train :", metrics.r2_score(y_train, gbrt.predict(X_train)))
    print("Score test:", metrics.r2_score(y_test, gbrt.predict(X_test)))
    print('MAE:', metrics.mean_absolute_error(np.expm1(y_test), np.expm1(predictions)))  
    print('MSE:', metrics.mean_squared_error(np.expm1(y_test), np.expm1(predictions)))
    print('RMSE:', np.sqrt(metrics.mean_squared_error(np.expm1(y_test), np.expm1(predictions))))
    print('MAPE:', metrics.mean_absolute_percentage_error(np.expm1(y_test), np.expm1(predictions)))

    gbrt.feature_importances_
    df_import_gbrt= pd.DataFrame({"col":X_train.columns, "coef_impor": gbrt.feature_importances_})
    df_import_gbrt= df_import_gbrt.round(decimals=2)
    df_import_gbrt.sort_values(by= "coef_impor",ascending= False).head(10)



6) Aplicamos PCA & Pipeline a los dos mejores modelos

    pipeline = Pipeline([('scaler',StandardScaler()),("pca",PCA(n_components=4)), ('regressor', GradientBoostingRegressor())])
    pipeline.fit(X_train, y_train)


    r2_train=pipeline.score(X_train, y_train)
    print(f"GBR_train: {r2_train}")

    r2 = pipeline.score(X_test, y_test)
    print(f"GBR_test: {r2}")

 7) Aplicamos pipeline y GridsearchCV a los dos mejores modelos  

    pipe = Pipeline(steps=[
        ("scaler", StandardScaler()),# este metodo sirve para escalar los datos 
        ('regressor', LinearRegression()) 
    ])


    bosting_ressor_params = {
        'regressor': [GradientBoostingRegressor()],
        'regressor__max_depth': [3,4,5,6,7,8],
        'regressor__n_estimators': [220,240,260,280]
    }


    search_space = [
    
        bosting_ressor_params,
    ]

    clf_b = GridSearchCV(estimator = pipe,
                    param_grid = search_space,
                    cv = 3)

    clf_b.fit(X_train, y_train)  

    from sklearn import metrics
    print(clf_b.best_estimator_)
    print(clf_b.best_score_) # 0,76
    print(clf_b.best_params_)
    
    print(" Results from Grid Search " )
    print("\n The best estimator across ALL searched params:\n",grid_XGB.best_estimator_)
    print("\n The best score across ALL searched params:\n",grid_XGB.best_score_)
    print("\n The best parameters across ALL searched params:\n",grid_XGB.best_params_)

8) GridSearchCV sin pipeline  al mejor modelo

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = seed)


    GBR = GradientBoostingRegressor()
    parameters = {'learning_rate': [0.01,0.02,0.03,0.04], #tasa_de_aprendizaje
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],#submuestra
                  'n_estimators' : [300,350,400],
                  'max_depth'    : [3,4,5,6,8,]# profundidad
                 }

    grid_GBR = GridSearchCV(estimator=GBR,  
                        param_grid = parameters,  # aqui van los parametros de los modelos utilizados
                        cv = 3,    #  cross validation 
                        n_jobs=-1) # Esto significa la cantidad de trabajos que se ejecutarán en paralelo, -1 significa usar todo el procesador.

    grid_GBR.fit(X_train, y_train) 


    print(" Results from Grid Search " )
    print("\n The best estimator across ALL searched params:\n",grid_GBR.best_estimator_)
    print("\n The best score across ALL searched params:\n",grid_GBR.best_score_)
    print("\n The best parameters across ALL searched params:\n",grid_GBR.best_params_)            